# 🔍 Sleep Data ML Training Debugger

This notebook helps debug why the ML training is showing 0 records and not reading sleep data properly.

In [ ]:
# Install required packages
!pip install supabase python-dotenv pandas numpy

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from supabase import create_client
import json
import warnings
warnings.filterwarnings('ignore')

print("✅ Packages imported successfully")

In [ ]:
# Configuration - UPDATE THESE WITH YOUR ACTUAL CREDENTIALS
SUPABASE_URL = "YOUR_SUPABASE_URL_HERE"
SUPABASE_KEY = "YOUR_SUPABASE_ANON_KEY_HERE"

print("🔑 Configuration set")
print(f"URL: {SUPABASE_URL[:30]}...")
print(f"Key: {SUPABASE_KEY[:20]}...")

In [ ]:
# Test Supabase connection
try:
    supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
    print("✅ Supabase client created successfully")
    
    # Test connection with a simple query
    response = supabase.table('sleep_records').select('*').limit(1).execute()
    print("✅ Database connection successful")
    print(f"📊 Sample query returned: {len(response.data)} records")
    
except Exception as e:
    print(f"❌ Connection failed: {e}")
    print("\n🔧 Check your credentials and network connection")

In [ ]:
# Check all tables in the database
try:
    # List all available tables (this might not work with all Supabase configurations)
    print("🔍 Checking database structure...")
    
    # Try to query each expected table
    "    tables_to_check = ['sleep_records', 'predictions']
",
    
    for table_name in tables_to_check:
        try:
            response = supabase.table(table_name).select('*').limit(1).execute()
            print(f"✅ Table '{table_name}' exists and accessible")
            
            # Get count
            count_response = supabase.table(table_name).select('*', count='exact').execute()
            record_count = len(count_response.data) if count_response.data else 0
            print(f"   📊 Records in '{table_name}': {record_count}")
            
            if record_count > 0:
                print(f"   📋 Sample record structure:")
                sample = response.data[0] if response.data else {}
                for key, value in sample.items():
                    print(f"      {key}: {type(value).__name__} = {str(value)[:50]}")
            
        except Exception as e:
            print(f"❌ Table '{table_name}' error: {e}")
        print()

except Exception as e:
    print(f"❌ Database structure check failed: {e}")

In [ ]:
# Detailed sleep_records analysis
try:
    print("🔍 Detailed sleep_records analysis...")
    
    # Get all sleep records
    response = supabase.table('sleep_records').select('*').order('start_time').execute()
    
    if not response.data:
        print("❌ NO SLEEP RECORDS FOUND!")
        print("\n🔧 Possible issues:")
        print("   1. Table is empty - no sleep data has been added")
        print("   2. Wrong table name - check if data is in different table")
        print("   3. Permission issues - check RLS policies")
        print("   4. Wrong credentials - verify Supabase URL and key")
    else:
        records = response.data
        print(f"✅ Found {len(records)} sleep records")
        
        # Convert to DataFrame for analysis
        df = pd.DataFrame(records)
        print(f"\n📊 DataFrame shape: {df.shape}")
        print(f"📋 Columns: {list(df.columns)}")
        
        # Data quality checks
        print("\n🔍 Data Quality Analysis:")
        print(f"   Missing start_time: {df['start_time'].isna().sum()}")
        print(f"   Missing end_time: {df['end_time'].isna().sum()}")
        
        if 'sleep_duration' in df.columns:
            print(f"   Missing sleep_duration: {df['sleep_duration'].isna().sum()}")
        
        # Date range
        df['start_time'] = pd.to_datetime(df['start_time'])
        print(f"   Date range: {df['start_time'].min()} to {df['start_time'].max()}")
        
        # Show sample records
        print("\n📋 Sample records:")
        print(df.head())
        
except Exception as e:
    print(f"❌ Sleep records analysis failed: {e}")

In [ ]:
# Test the exact query used by the ML training API
try:
    print("🧪 Testing exact ML training query...")
    
    # This is the exact query from the train-model API
    response = supabase.table("sleep_records").select("*").order("start_time", desc=False).execute()
    
    if response.data:
        print(f"✅ ML training query successful: {len(response.data)} records")
        
        # Check if we have minimum records for training
        if len(response.data) >= 30:
            print(f"✅ Sufficient data for ML training ({len(response.data)} >= 30)")
        else:
            print(f"⚠️  Insufficient data for ML training ({len(response.data)} < 30)")
            print("   Need at least 30 records for training")
        
        # Show data types and structure
        sample_record = response.data[0]
        print("\n📋 Record structure (exactly as API sees it):")
        for key, value in sample_record.items():
            print(f"   {key}: {type(value).__name__} = {value}")
            
    else:
        print("❌ ML training query returned 0 records")
        print("\n🔧 This is exactly why your ML training shows 0 records!")
        
except Exception as e:
    print(f"❌ ML training query failed: {e}")
    print(f"   Error details: {str(e)}")

In [ ]:
# Check RLS (Row Level Security) policies
try:
    print("🔐 Checking Row Level Security...")
    
    # Try different ways to access the data
    queries_to_test = [
        ("Basic select", lambda: supabase.table('sleep_records').select('*').execute()),
        ("With limit", lambda: supabase.table('sleep_records').select('*').limit(5).execute()),
        ("Count only", lambda: supabase.table('sleep_records').select('*', count='exact').execute()),
        ("Select specific columns", lambda: supabase.table('sleep_records').select('id,start_time,end_time').execute())
    ]
    
    for query_name, query_func in queries_to_test:
        try:
            result = query_func()
            record_count = len(result.data) if result.data else 0
            print(f"   ✅ {query_name}: {record_count} records")
        except Exception as e:
            print(f"   ❌ {query_name}: {e}")
            
except Exception as e:
    print(f"❌ RLS check failed: {e}")

In [ ]:
# Create sample data if no records exist
try:
    print("🔧 Creating sample data for testing...")
    
    # Check current record count
    current_response = supabase.table('sleep_records').select('*').execute()
    current_count = len(current_response.data) if current_response.data else 0
    
    if current_count == 0:
        print("📝 No existing records - creating sample data...")
        
        # Generate 50 sample sleep records
        sample_records = []
        base_date = datetime.now() - timedelta(days=60)
        
        for i in range(50):
            # Random sleep patterns
            sleep_start = base_date + timedelta(days=i)
            sleep_start = sleep_start.replace(hour=22 + np.random.randint(-2, 3), minute=np.random.randint(0, 60))
            
            # Sleep duration between 6-9 hours
            duration_hours = 6 + np.random.random() * 3
            sleep_end = sleep_start + timedelta(hours=duration_hours)
            
            record = {
                'start_time': sleep_start.isoformat(),
                'end_time': sleep_end.isoformat(),
                'sleep_duration': duration_hours,
                'sleep_quality': np.random.randint(1, 6),
                'stress_level': np.random.randint(1, 6),
                'caffeine_intake': np.random.randint(0, 3),
                'exercise_hours': np.random.random() * 2,
                'sleep_cycle': np.random.randint(3, 7)
            }
            sample_records.append(record)
        
        # Insert sample records
        insert_response = supabase.table('sleep_records').insert(sample_records).execute()
        
        if insert_response.data:
            print(f"✅ Created {len(insert_response.data)} sample records")
            print("🎉 You can now test ML training with sample data!")
        else:
            print("❌ Failed to create sample records")
            print("   Check table permissions and schema")
    else:
        print(f"✅ Already have {current_count} records - no sample data needed")
        
except Exception as e:
    print(f"❌ Sample data creation failed: {e}")
    print("   This might be due to RLS policies or schema mismatches")

In [ ]:
# Final verification
try:
    print("🎯 Final verification...")
    
    # Re-check record count after potential sample data creation
    final_response = supabase.table('sleep_records').select('*').order('start_time').execute()
    final_count = len(final_response.data) if final_response.data else 0
    
    print(f"📊 Total sleep records: {final_count}")
    
    if final_count >= 30:
        print("✅ Ready for ML training!")
        print("🚀 You can now run the ML training in your web app")
        
        # Show what the ML training will see
        print("\n📋 Data summary for ML training:")
        df = pd.DataFrame(final_response.data)
        df['start_time'] = pd.to_datetime(df['start_time'])
        print(f"   Date range: {df['start_time'].min().date()} to {df['start_time'].max().date()}")
        print(f"   Average sleep duration: {df['sleep_duration'].mean():.1f} hours")
        print(f"   Records per week: {final_count / ((df['start_time'].max() - df['start_time'].min()).days / 7):.1f}")
        
    elif final_count > 0:
        print(f"⚠️  Need {30 - final_count} more records for ML training")
        print("📝 Add more sleep data or create additional sample records")
    else:
        print("❌ Still no sleep records found")
        print("\n🔧 Next steps:")
        print("   1. Check your Supabase credentials")
        print("   2. Verify table name and schema")
        print("   3. Check RLS policies")
        print("   4. Add sleep data through your web app")
        
except Exception as e:
    print(f"❌ Final verification failed: {e}")

## 🎯 Summary

This notebook helps you diagnose why ML training shows 0 records. Common issues:

1. **No Data**: Table is empty - add sleep records first
2. **Wrong Credentials**: Check Supabase URL and API key
3. **RLS Policies**: Row Level Security blocking access
4. **Schema Mismatch**: Table structure doesn't match expected format
5. **Network Issues**: Connection problems to Supabase

Run all cells above to identify and fix the issue!